<h2>Importing Data and Librarires</h2>

In [1]:
import numpy as np
import pandas as pd
from future.utils import iteritems
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score
import sys

if not sys.warnoptions: # ignore warnings 3ahsan e7na rewsheen gedan
    import warnings
    warnings.simplefilter("ignore")

In [2]:
whr15 = pd.read_csv('2015.csv')
whr16 = pd.read_csv('2016.csv')
whr17 = pd.read_csv('2017.csv')
gtd17 = pd.read_csv('globalterrorismdb_0718dist.csv',encoding='iso-8859-1',low_memory=False)

In [3]:
gtd17.head(1)


eventid  iyear  imonth  iday approxdate  extended resolution  country  \
0  197000000001   1970       7     2        NaN         0        NaN       58   

          country_txt  region   ...    addnotes scite1 scite2  scite3  \
0  Dominican Republic       2   ...         NaN    NaN    NaN     NaN   

   dbsource  INT_LOG  INT_IDEO INT_MISC INT_ANY  related  
0      PGIS        0         0        0       0      NaN  

[1 rows x 135 columns]

<h2>Preprocessing GTD only</h2>

In [4]:
# MENA regions only
ds = gtd17[gtd17['region_txt'] == 'Middle East & North Africa']
# Year 2000-2017
ds = ds[ds['iyear'] > 1999]
# removing incidents where group is unknown
ds = ds[ds['gname'] != 'Unknown']
# choosing top groups
votes = {}
for name in ds['gname']:
    votes[name] = votes.get(name,0)+1

counts = []
names = []
for name,count in iteritems(votes):
    names.append(name)
    counts.append(count)
    
names = [name for _,name in sorted(zip(counts,names))]
names = names[-5:]
contained = [name in names for name in ds['gname']]
ds = ds[contained]
# encoding gname
lb_make = LabelEncoder()
ds["gname"] = lb_make.fit_transform(ds["gname"])
# drop redundant col , for ex region and region_txt , drop region
names = list(set(ds))
drop = ['region','region_txt','eventid','gname']
for col in names:
    f = str(col)+'_txt'
    if (f) in names and f not in drop :
            drop.append(f) # for ex if 


<h2>Split train: incidents 2000-2013 , test: incidents 2014-2017</h2>

In [5]:
Xtrain = ds[ds['iyear']<2014]
Ytrain = Xtrain['gname']
Xtrain = Xtrain.drop(columns=drop)
Xtest = ds[ds['iyear']>2013]
Ytest = Xtest['gname']
Xtest = Xtest.drop(columns=drop)


<h2>Getting best features</h2>

In [6]:
votes = {}
for name in Ytrain:
    votes[name] = votes.get(name,0)+1

print('number of attakcs per each group in train ' ,votes)
votes = {}
for name in Ytest:
    votes[name] = votes.get(name,0)+1

print('number of attakcs per each group in test ' ,votes)

number of attakcs per each group in train  {4: 311, 0: 637, 1: 470, 2: 35, 3: 374}
number of attakcs per each group in test  {3: 5189, 1: 547, 2: 1027, 4: 912}


In [7]:
def get_features(model , Xtrain , Ytrain):
    num = len(Xtrain.columns) # total number of features
    best_scores = []
    names = list(Xtrain.columns) # names of all features
    Y = Ytrain 
    best_scores = []
    good_features = [] # stores the names of the best selected features
    bad_features = [] # stores the names of the  bad features not to try with again
    for i in range(5) : # take best 5 features
        scores = []
        for j,col in enumerate(Xtrain): # search for the next best feature that is the one which increases accuracy the most
            percent = round((i*104+j)/(104*5)*100,0)
            print('Loading '+str(percent)+' %',end="\r")
           
            if col in good_features or col in bad_features: # if feature reviewed before as good or bad skip
                scores.append(0)
                continue
            feature = Xtrain[col]  # get each feature
            X = Xtrain[good_features]  # get previous selected features
            #X.loc[:,col] = feature   # add this feature
            X = X.assign(col=feature)
            if 'O' in list(set(X.dtypes)): # one of the features is not numeric and needs encoding , note that nan values become zeros
                X = pd.get_dummies(X)
            X = X.fillna(X.mean()) # replace nan values by mean in numeric features
            if len (X.columns) > 20: # one-hot encoding results in too many dimensions , skip
                scores.append(0)
                continue
            if len (X.columns) == 0: # dont know why this is here
                scores.append(0)
                continue
            if X.isnull().values.any(): # a feauture still has nan values , this occurs if it initially has only nan values , it may had values in the removed columns
                scores.append(0)
                continue
           
            model.fit(X,Y)

            score = model.score(X,Y)
            if score < 0.02: # features that score less than 2% are bad and should not be reconsidered
                scores.append(0)
                bad_features.append(col)
            else:
                scores.append(score)
                
        index = scores.index(max(scores)) # get best feature
        best_scores.append(max(scores)) # store best score for 1 to n features
        good_features.append(names[index])
    return (good_features,best_scores)

In [8]:
def get_score(model,good_features,ds): # return the score after selecting best features
    scores = []
    fscores = []
    for i in range(5):
        features=good_features.copy()
        features = features[:i+1] 
        selected = features.copy()
        if 'iyear' not in features:
            selected.append('iyear')
        selected.append('gname')
        df = ds[selected]
        df = pd.get_dummies(df)
        df = df.fillna(df.mean())

        Xtrain = df[df['iyear']<2014]
        Ytrain = Xtrain['gname']
        Xtrain = Xtrain.drop(columns=['iyear','gname']) # always drop iyear , since test and trai wikk akways have different years
        Xtest = df[df['iyear']>2013]
        Ytest = Xtest['gname']
        Xtest = Xtest.drop(columns=['iyear','gname'])
        model.fit(Xtrain,Ytrain)
        scores.append(model.score(Xtest,Ytest))
        fscores.append(f1_score(Ytest,model.predict(Xtest),average='weighted'))
    return (max(scores),max(fscores))

<h2>KNN</h2>

In [9]:
model = KNeighborsClassifier(n_neighbors=2)
good_features,best_scores = get_features(model,Xtrain,Ytrain)
print('good features ',good_features)
print('train scores :',best_scores)
score,fscore = get_score(model,good_features,ds)
print('test score ',score)
print('test fscore ',fscore)

good features  ['latitude', 'iyear', 'INT_IDEO', 'country', 'INT_MISC']
train scores : [0.8286808976464148, 0.9293924466338259, 0.9917898193760263, 0.9967159277504105, 0.9983579638752053]
test score  0.9401954397394137
test fscore  0.9496596568949316


<h2>Decision Trees</h2>

In [10]:
model = DecisionTreeClassifier()
good_features,best_scores = get_features(model,Xtrain,Ytrain)
print('good features ',good_features)
print('train scores :',best_scores)
score,fscore = get_score(model,good_features,ds)
print('test score ',score)
print('test fscore ',fscore)

good features  ['latitude', 'INT_IDEO', 'iyear', 'imonth', 'iday']
train scores : [0.8538587848932676, 0.9813902572523262, 0.9983579638752053, 1.0, 1.0]
test score  0.9252117263843648
test fscore  0.9427029465197565


<h2>Naive Bayes</h2>

In [11]:
model = GaussianNB() 
good_features,best_scores = get_features(model,Xtrain,Ytrain)
print('good features ',good_features)
print('train scores :',best_scores)
score,fscore = get_score(model,good_features,ds)
print('test score ',score)
print('test fscore ',fscore)

good features  ['country', 'INT_IDEO', 'ransomnote', 'nwoundus', 'INT_LOG']
train scores : [0.7679255610290093, 0.9496442255062945, 0.9704433497536946, 0.9753694581280788, 0.9775588396278051]
test score  0.9696416938110749
test fscore  0.9703578358822619


<h2>SVM</h2>

In [13]:
model = LinearSVC() 
good_features,best_scores = get_features(model,Xtrain,Ytrain)
print(good_features)
print(best_scores)
score,fscore = get_score(model,good_features,ds)
print('test score ',score)
print('test fscore ',fscore)

['latitude', 'INT_IDEO', 'natlty1', 'nwoundus', 'nkillter']
[0.7066228790366721, 0.9458128078817734, 0.9660645867542419, 0.9671592775041051, 0.9693486590038314]
test score  0.9649511400651466
test fscore  0.9648798561978983
